# Semantical

## AMR parsing

- 常用公開資料集: [LDC2017T10](https://catalog.ldc.upenn.edu/LDC2017T10)
> - 跨語對應資料集: [LDC2020T07](https://catalog.ldc.upenn.edu/LDC2020T07) & [	LDC2014T12](https://catalog.ldc.upenn.edu/LDC2014T12) (中/德/義/西)

-----------

### [JAMR-Chinese](https://github.com/tanglrHello/nlpnju)

- 老師找到的
- 專門做中文的，資料集也是 little prince


### [XL-AMR](https://github.com/SapienzaNLP/xl-amr)

- **Input**: 中/德/義/西文 sentences | **Output**: 英文 AMR graph
- English AMR transfer learning for 中、德、義、西文
- 有提供 pretrained models [CHECKPOINTS](https://drive.google.com/drive/folders/1_tu6EJET20pi5IG3T-807hDpBjtkPs7W)

- 兩種訓練策略—
> 1. Parallel sentences - silver AMR graphs 
>> (見論文：Experiments-Dataset、Table 1)<br>
>> 中文沒有採用這個方法，因為 Europarl 裡沒有中英 parallel corpus，但中英翻譯 parallel corpus 應該不難找，可以自己 try。
> 2. Gold AMR graphs - silver translations
>> - 用 AMR 2.0 (dataset) 
>>> - 以 OPUS-MT[3] (pre-trained 翻譯模型) 訓練 英→德|義|西文翻譯
>>> - 以 MASS[4] (pre-trained 翻譯模型) 訓練 英→中文翻譯

- 中文因為 **Europarl**[1] 沒有 parallel corpus 所以沒有 amr graph 的 silver data
- 如何產生 silver dataset (中德義西):
> 1. 以 AMR 演算法[2] 生成 Europarl[1] (翻譯資料集) 的 英文部分的 AMR graph (沒有中文)
> 2. 利用 pretrained 翻譯模型--OPUS-MT[3] 翻譯 AMR 2.0(LDC2017T10) 的句子

> - [1]**Parallel Corpus 來源: Europarl**
>> *Philipp Koehn. 2005. Europarl: A Parallel Corpus for Statistical Machine Translation. In Conference Proceedings: the tenth Machine Translation Summit, pages 79–86, Phuket, Thailand. AAMT, AAMT.*

> - [2]**AMR 演算法: [AMR Parsing as Sequence-to-Graph Transduction](https://github.com/sheng-z/stog)** (!)
>> *Sheng Zhang, Xutai Ma, Kevin Duh, and Benjamin Van Durme. 2019. AMR Parsing as Sequence-to-Graph Transduction. In Proceedings of the 57th Annual Meeting of the Association for Computational Linguistics, pages 80–94, Florence, Italy. Association for Computational Linguistics.*

> - [3]**OPUS-MT pretrained models**
>> https://huggingface.co/docs/transformers/model_doc/marian

> - [4]**MASS pretrained model** (!training-only)
>> https://github.com/microsoft/MASS/tree/master/MASS-supNMT <br>
>> 有 code, 有 model, 讚讚


In [ ]:
# 當前目標以預測為主，先不重新訓練 (notice: 輸出結果為 英文 AMR graph)
# 預測步驟:
# 1. 下載中文預訓練模型 https://drive.google.com/drive/folders/1iIQyfb-HdLL4VKhwe-n0aZ7Zv7yRtbry
# 2. 

# 目前執行環境--Anaconda

In [ ]:
# 0.00024461839348077774 % 5
import torch
a = torch.tensor([1, 2, 3, 4, 5])
b = -1.5
print(torch.fmod(a, b))
print(a.div(b, rounding_mode="trunc"))
print(a.div(b, rounding_mode="trunc") * b)
print(a - a.div(b, rounding_mode="trunc") * b)

##### 執行過程相關問題
- java 8 (done)
- 輸入格式錯誤 (done)
> 因為沒有加入 preprosses features
- json quote (done)
- [有人建議改用 stanza](https://blog.csdn.net/AdrenalinmyVein/article/details/124896877)
- indices/beam_indices 要改型態為 long (ex: indices.long()) (改long 怪怪的，再研究beam search)
> *參考第三點網址*
- torch.div 要加 rounding_mode="trunc" 才會四捨五入 (一樣怪怪的，再研究beam search)
- 輸入最大 token 長度 (2) 小於 pretrained model 的 kernel size (3)，從輸入資料修正即可，再考慮是否防呆 (done)


--------

### [STOG (AMR Parsing as Sequence-to-Graph Transduction)](https://github.com/sheng-z/stog)

- XLAMR 取得英文 AMR silver data 採用的模型

In [ ]:
## 測試，執行在 local conda env 'stog'
import json

fr = open("config.json", "r", encoding="utf-8").read()
frjson = json.loads(fr)
frjson

{'amr_data_dir': 'data/amr_2.0',
 'data': {'batch_first': True,
  'data_dir': 'data/amr_2.0',
  'data_type': 'AMR',
  'dev_data': 'dev_amr.txt.features.preproc',
  'iter_type': 'BucketIterator',
  'pretrain_char_emb': None,
  'pretrain_token_emb': '/export/ssd/sheng/data/glove/glove.840B.300d.zip',
  'test_batch_size': 32,
  'test_data': 'test_amr.txt.features.preproc',
  'train_batch_size': 64,
  'train_data': 'train_amr.txt.features.preproc',
  'word_splitter': 'data/bert-base-cased/bert-base-cased-vocab.txt'},
 'environment': {'cuda_device': 0,
  'file_friendly_logging': False,
  'gpu': True,
  'numpy_seed': 1,
  'recover': False,
  'seed': 1,
  'serialization_dir': 'stog-bert-ckpt',
  'torch_seed': 1},
 'glove': '/export/ssd/sheng/data/glove/glove.840B.300d.zip',
 'model': {'bert': {'hidden_size': 768,
   'pretrained_model_dir': 'data/bert-base-cased'},
  'coref_attention': {'attention_function': 'mlp', 'share_linear': True},
  'decoder': {'dropout': 0.33,
   'hidden_size': 1024,
 

### [AMRLIB](https://github.com/bjascob/amrlib-models)
- [參考自 Simple XAMR](https://github.com/Heidelberg-NLP/simple-xamr)
> (目前理解) XAMR 步驟：先 以 [EasyNMT](https://github.com/UKPLab/EasyNMT) 將義、西、得、中文 ([LDC2020T07](https://catalog.ldc.upenn.edu/LDC2020T07) 資料集) 翻譯成 英文；然後再用 [amrlib](https://github.com/bjascob/amrlib) 把翻譯完的結果 parse 成 (英文) AMR graphs。

- 現成的 pretrained model 只有英文 ([說明](https://github.com/bjascob/amrlib-models#sentence-to-graph-models) : "*All models are trained and scored on AMR-3 ([LDC2020T02](https://paperswithcode.com/dataset/ldc2020t02))*" ) ([released models](https://github.com/bjascob/amrlib-models/releases))，但可以自己拿dataset train ([AMRLib--For Training](https://amrlib.readthedocs.io/en/latest/install/))

#### Prepare environment

In [ ]:
%%shell

pip install amrlib

#### Main

In [ ]:
import amrlib

--------------

### [Transition-based AMR Parser](https://github.com/IBM/transition-amr-parser)
- 融易找到的，但得寫信給作者拿 checkpoint
- IBM EMNLP 2020 的[論文](https://arxiv.org/pdf/2010.10669.pdf)
- *Abstract Meaning Representation Guided Graph Encoding and Decoding
for Joint Information Extraction* 所採用的 AMR parser

------------

### [amr-eager](https://github.com/mdtux89/amr-eager-multilingual#Parse)
- [data](https://github.com/mdtux89/amr-eager-multilingual/blob/master/download-data.sh)
- [model](https://github.com/mdtux89/amr-eager-multilingual/blob/master/download.sh)

#### Prepare environment

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

In [ ]:
%%shell
git clone https://github.com/jflanigan/jamr.git
git checkout Semeval-2016
pip install mafan

In [ ]:
!git clone https://github.com/mdtux89/amr-eager-multilingual.git

In [ ]:
!wget https://nlp.stanford.edu/software/stanford-corenlp-latest.
!unzip stanford-corenlp-latest.zip
!wget https://search.maven.org/remotecontent?filepath=edu/stanford/nlp/stanford-corenlp/4.4.0/stanford-corenlp-4.4.0-models-chinese.jar

In [ ]:
!java -version

In [ ]:
!git lfs install
!git clone https://huggingface.co/stanfordnlp/corenlp-chinese

#### Main

In [ ]:
!amr-eager-multilingual/preprocessing.sh -s -l zh -f shiji_sample.txt

In [ ]:
%%shell
java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-chinese.properties -port 9000 -timeout 15000

##### 從 sh 檔中擷取出問題的指令出來測試：

測試 1.
| [相似問題](https://bytemeta.vip/repo/stanfordnlp/CoreNLP/issues/1098)

In [ ]:
!java -Xmx2500m -cp "/content/stanford-corenlp-4.4.0/stanford-corenlp-4.4.0.jar" edu.stanford.nlp.pipeline.StanfordCoreNLP -props "chinese.properties" -file "./shiji_sample.txt.sentences" -outputFormat text -replaceExtension --outputDirectory "./"

In [ ]:
!java -mx9g -cp "/content/stanford-corenlp-4.4.0/*" edu.stanford.nlp.pipeline.StanfordCoreNLP -props "chinese.properties" -file "./shiji_sample.txt" -outputFormat text -replaceExtension --outputDirectory "./"

測試 2. 
| [相似問題](https://github.com/smilli/py-corenlp/issues/11)

In [ ]:
!java -mx3g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -props "chinese.properties" -file "./shiji_sample.txt.sentences" -outputFormat text -replaceExtension --outputDirectory "./"

In [ ]:
!java -mx3g -cp "stanford-corenlp-4.4.0/*" edu.stanford.nlp.pipeline.StanfordCoreNLP -outputFormat json -file shiji_sample.txt

[問題3](https://github.com/stanfordnlp/stanza/issues/12#issuecomment-477182270)

In [ ]:
!unzip /content/stanford-corenlp-4.4.0-models-chinese.jar

In [ ]:
%%shell
cd /content/edu/stanford/nlp/models/
gzip -d */*/*.gz

In [ ]:
!jar cf stanford-corenlp-4.4.0-models-chinese.jar /content/StanfordCoreNLP-chinese.properties /content/edu/ /content/META-INF/

In [ ]:
# !mv stanford-corenlp-4.4.0-models-chinese.jar /content/stanford-corenlp-4.4.0/
!cp /content/stanford-corenlp-4.4.0/stanford-corenlp-4.4.0-models-chinese.jar /content/drive/MyDrive/IISR/Work/111Plan/

-------------

### [Chinese Abstract Meaning Representation](https://www.cs.brandeis.edu/~clp/camr/camr.html)
- Python 2.7 + Java 1.8
- [Github](https://github.com/c-amr/camr)
> - Github 上沒有提到中文模型，照文件來看使用的是英文資料集 [LDC2014T12](https://catalog.ldc.upenn.edu/LDC2014T12) & [SemEval2016](https://alt.qcri.org/semeval2016/task8/)，[論文](https://aclanthology.org/W16-1702.pdf) 也只提到有釋出中文 AMR 資料集，沒有提到模型或程式碼 (看起來是共用 CAMR github)。

In [ ]:
# ! git clone https://github.com/Juicechuan/AMRParsing.git
!pip install 2to3
!2to3 --write --nobackups AMRParsing/

In [ ]:
! wget https://nlp.stanford.edu/software/stanford-corenlp-latest.zip
! unzip stanford-corenlp-latest.zip
! cp -R stanford-corenlp-4.4.0/ AMRParsing/stanfordnlp/

In [ ]:
! rm -R stanford-corenlp-4.4.0/

In [ ]:
# ! wget http://nlp.stanford.edu/software/stanford-corenlp-full-2015-04-20.zip
# ! unzip stanford-corenlp-full-2015-04-20.zip

In [ ]:
! pip install --user bllipparser

In [ ]:
!java -Xmx2500m -version

好像只能用第8版

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

In [ ]:
# ! wget https://search.maven.org/remotecontent?filepath=edu/stanford/nlp/stanford-corenlp/4.4.0/stanford-corenlp-4.4.0-models-chinese.jar
# ! unzip stanford-corenlp-4.4.0-models-chinese.zip
# ! mv "./stanford-corenlp-4.4.0-models-chinese.jar" "AMRParsing/stanfordnlp/stanford-corenlp-4.4.0/"

In [ ]:
! python AMRParsing/amr_parsing.py -m preprocess chinese-onesent-unseg-utf8.txt

-------------

## Dependency parsing

# Syntactical 

## 綜合工具總整理

### [綜合工具--Stanza](https://stanfordnlp.github.io/stanza/)
- POS Tagging
- NER
- Lemmas
- Dependency parsing
- Constituency parsing
> ---
- 
基本 NLP 模型、API 一應俱全，經常被 AMR 等依賴相關特徵的任務作為前處理工具，也有 Classical Chinese 模型，但用史記在線上測試輸出結果怪怪的。

- [線上測試](http://stanza.run/)
- [Stanford CoreNLP](https://stanfordnlp.github.io/CoreNLP/download.html) |  [Github](https://github.com/stanfordnlp/CoreNLP)

##### set environment

In [ ]:
! pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 4.4 MB/s 
     |████████████████████████████████| 197 kB 58.1 MB/s 
     |████████████████████████████████| 4.4 MB 31.0 MB/s 
     |████████████████████████████████| 596 kB 22.3 MB/s 
     |████████████████████████████████| 101 kB 3.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.3 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=3953adf785d4eb530fc562ccfa11abdbf6689f2514073f7b9492f00c9f362c7b
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##### main

In [ ]:
import stanza

## 指定 stanza functions
stanza.download('zh')
nlp = stanza.Pipeline('zh', processors='tokenize,pos,lemma,depparse,ner,constituency')

2022-07-16 23:56:32 INFO: "zh" is an alias for "zh-hans"
2022-07-16 23:56:32 INFO: Downloading default packages for language: zh-hans (Simplified_Chinese)...


2022-07-16 23:57:31 INFO: Finished downloading models and saved to /root/stanza_resources.


2022-07-16 23:57:31 INFO: "zh" is an alias for "zh-hans"
2022-07-16 23:57:32 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package   |
----------------------------
| tokenize     | gsdsimp   |
| pos          | gsdsimp   |
| lemma        | gsdsimp   |
| depparse     | gsdsimp   |
| constituency | ctb       |
| ner          | ontonotes |

2022-07-16 23:57:32 INFO: Use device: cpu
2022-07-16 23:57:32 INFO: Loading: tokenize
2022-07-16 23:57:33 INFO: Loading: pos
2022-07-16 23:57:33 INFO: Loading: lemma
2022-07-16 23:57:33 INFO: Loading: depparse
2022-07-16 23:57:33 INFO: Loading: constituency
2022-07-16 23:57:34 INFO: Loading: ner
2022-07-16 23:57:35 INFO: Done loading processors!


In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,depparse,ner,constituency')

2022-07-16 23:57:59 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| constituency | wsj       |
| ner          | ontonotes |

2022-07-16 23:57:59 INFO: Use device: cpu
2022-07-16 23:57:59 INFO: Loading: tokenize
2022-07-16 23:57:59 INFO: Loading: pos
2022-07-16 23:57:59 INFO: Loading: lemma
2022-07-16 23:57:59 INFO: Loading: depparse
2022-07-16 23:57:59 INFO: Loading: constituency
2022-07-16 23:58:00 INFO: Loading: ner
2022-07-16 23:58:01 INFO: Done loading processors!


In [ ]:
a = [[0,1], [2]]
b = []
for i in a :
    if len(b) == 0 :
        b = i
    else :
        b = b + i
b

[0, 1, 2]

In [ ]:
## 批次轉 stanza 輸出 to xlamr 輸入格式

input_file = open("input.txt", "r", encoding="utf-8").read()
input_sents = input_file.split("\n")
sent_dummy = ""

for i, sent in enumerate(input_sents) :
    print(sent)
    output_dict = {"tokens":[], "lemmas":[], "pos_tags":[], "ner_tags":[]}
    sent_id = str(i+1)

    if len(sent) > 0 :
        sent_dummy = sent_dummy + "# ::id sents_id."+ sent_id + "\n# ::snt " + sent + "\n"

        doc = nlp(sent)
        sent_dicts = doc.to_dict()

        ### EN VER (zh testing) ###
        sent_dicts_ = []
        for i in sent_dicts :
            if len(sent_dicts_) == 0 :
                sent_dicts_ = i
            else :
                sent_dicts_ = sent_dicts_ + i
        
        ### ZH VER (old) ###
        # snet_dicts_ = sent_dicts[0]

        for word_dict in sent_dicts_ :
            output_dict["tokens"].append(word_dict["text"])
            output_dict["lemmas"].append(word_dict["lemma"])
            output_dict["pos_tags"].append(word_dict["xpos"])
            output_dict["ner_tags"].append(word_dict["multi_ner"][0])

        for feature in output_dict :
            write_txt = "# ::" + feature + " " + str(output_dict[feature])
            sent_dummy = sent_dummy + write_txt + "\n"

        sent_dummy = sent_dummy + "(d / dummy)\n\n"
        while "'" in sent_dummy :
            sent_dummy = sent_dummy.replace("'", '"')
        while '""""' in sent_dummy :
            sent_dummy = sent_dummy.replace('""""', '""')


In [ ]:
fo = open("stanza_output.txt", "w", encoding="utf-8")
fo.write(sent_dummy)
fo.close()

In [ ]:
print("output_dict>>>\n", output_dict, "\n")
print(sent_dummy)

output_dict>>>
 {'tokens': [], 'lemmas': [], 'pos_tags': [], 'ner_tags': []} 

# ::id sents_id.1
# ::snt Tai Shigong said: The records of the five emperors and the three generations are still pending.
# ::tokens ["Tai", "Shigong", "said", ":", "The", "records", "of", "the", "five", "emperors", "and", "the", "three", "generations", "are", "still", "pending", "."]
# ::lemmas ["Tai", "Shigong", "say", ":", "the", "record", "of", "the", "five", "emperor", "and", "the", "three", "generation", "be", "still", "pend", "."]
# ::pos_tags ["NNP", "NNP", "VBD", ":", "DT", "NNS", "IN", "DT", "CD", "NNS", "CC", "DT", "CD", "NNS", "VBP", "RB", "VBG", "."]
# ::ner_tags ["B-PERSON", "E-PERSON", "O", "O", "O", "O", "O", "O", "S-CARDINAL", "O", "O", "O", "S-CARDINAL", "O", "O", "O", "O", "O"]
(d / dummy)

# ::id sents_id.2
# ::snt Since the feudal lords before Yin were not available and recorded, it has been quite available since Zhou.
# ::tokens ["Since", "the", "feudal", "lords", "before", "Yin", "were

In [ ]:
from torch import float32
import torch
t1 = torch.tensor([[2.0039e+00, 4.3767e-03, 4.0007e+00, 1.0017e+00, 7.2945e-04],
        [1.6615e-02, 4.8630e-04, 2.0166e+00, 1.0166e+00, 3.0166e+00],
        [4.0496e+00, 2.4315e-03, 4.9603e-02, 5.4304e-03, 4.0525e-04],
        [1.0496e+00, 1.3365e+00, 4.0496e+00, 8.2996e-02, 2.4315e-03]], dtype=float32)
t2 = t1.long()
t3 = t1.to(torch.int64)
print(t2)
print(t3)

-----------

### [綜合工具--CKIP **CoreNLP**](https://github.com/ckiplab/ckipnlp)
- Sentence Segmentation
- Word Segmentation
- Part-of-Speech Tagging
- Named-Entity Recognition
- Constituency Parsing
- Coreference Resolution
> ---
- [線上測試](https://ckip.iis.sinica.edu.tw/service/corenlp/)


##### set environment

In [ ]:
# !pip install ckipnlp
# !pip install ckipnlp[tagger]
!pip install ckipnlp[classic]

-------------------

### [綜合工具--CKIP **Han Transformers**](https://github.com/ckiplab/han-transformers)

沒有 NER 但有 word segmentation，也有 pre-trained LM 可以拿去 ckip-transformer fine-tune
- Pre-trained LM
- Tokenizer
- POS tagging
- Word segmentation
> ---
- Han Transformers 是 CKIP 的古漢語 (上古/中古/近代/現代) Transformer-based CoreNLP toolkits 
- [hugging_face](https://huggingface.co/ckiplab/bert-base-han-chinese)

##### set environment

In [3]:
%%shell
pip install transformers==4.15.0
pip install torch==1.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 14.1 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 3.3 MB 55.3 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 880 kB 77.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=317e74500570e19ead1749235f4c516f8556c64c62dbd504c0fbb3eeb316f4e1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 776.3 MB 21 kB/s 
  Attemp

Load pretrained model

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Initialize tokenzier and model
tokenizer = AutoTokenizer.from_pretrained("ckiplab/bert-base-han-chinese")
model = AutoModel.from_pretrained("ckiplab/bert-base-han-chinese")

# Input text
text = "黎民於變時雍。"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

# get encoded token vectors
output.last_hidden_state    # torch.Tensor with Size([1, 9, 768])

# get encoded sentence vector
output.pooler_output        # torch.Tensor with Size([1, 768])

Pipeline for token classification (pos)

In [4]:
import transformers
from transformers import pipeline, PreTrainedTokenizer

# Initialize
classifier = pipeline("token-classification", model="ckiplab/bert-base-han-chinese-pos")

# Input text
text = "靈公立十四年，益驕。趙盾驟諫，靈公弗聽。及食熊蹯，胹不熟，殺宰人，持其屍出，趙盾見之。靈公由此懼，欲殺盾。"
classifier(text)

Downloading:   0%|          | 0.00/18.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'entity': 'NB1',
  'score': 0.998198,
  'index': 1,
  'word': '靈',
  'start': 0,
  'end': 1},
 {'entity': 'NB1',
  'score': 0.998425,
  'index': 2,
  'word': '公',
  'start': 1,
  'end': 2},
 {'entity': 'VH1',
  'score': 0.8487876,
  'index': 3,
  'word': '立',
  'start': 2,
  'end': 3},
 {'entity': 'S',
  'score': 0.9940771,
  'index': 4,
  'word': '十',
  'start': 3,
  'end': 4},
 {'entity': 'S',
  'score': 0.9901356,
  'index': 5,
  'word': '四',
  'start': 4,
  'end': 5},
 {'entity': 'NA5',
  'score': 0.99941504,
  'index': 6,
  'word': '年',
  'start': 5,
  'end': 6},
 {'entity': 'COMMACATEGORY',
  'score': 0.9985468,
  'index': 7,
  'word': '，',
  'start': 6,
  'end': 7},
 {'entity': 'DF',
  'score': 0.96427774,
  'index': 8,
  'word': '益',
  'start': 7,
  'end': 8},
 {'entity': 'VH1',
  'score': 0.9870613,
  'index': 9,
  'word': '驕',
  'start': 8,
  'end': 9},
 {'entity': 'PERIODCATEGORY',
  'score': 0.9976053,
  'index': 10,
  'word': '。',
  'start': 9,
  'end': 10},
 {'entity': 

In [7]:
import json

tokens = classifier(text)
# tokens_ = {"text": text, "tokens": tokens}
tokens_ = str(tokens).replace("'", '"')
tokens = json.loads(tokens_)
print(tokens_)
# for i, token in enumerate(tokens) :
#   for t in token :
#     if "numpy" in str(type(token[t])) :
#       print(token[t])
#       tokens[i][t] = float(token[t])
      
with open("output.json", "w", encoding="utf-8") as fw :
  json.dump(tokens, fw, indent=4, ensure_ascii=False)


[{"entity": "NB1", "score": 0.998198, "index": 1, "word": "靈", "start": 0, "end": 1}, {"entity": "NB1", "score": 0.998425, "index": 2, "word": "公", "start": 1, "end": 2}, {"entity": "VH1", "score": 0.8487876, "index": 3, "word": "立", "start": 2, "end": 3}, {"entity": "S", "score": 0.9940771, "index": 4, "word": "十", "start": 3, "end": 4}, {"entity": "S", "score": 0.9901356, "index": 5, "word": "四", "start": 4, "end": 5}, {"entity": "NA5", "score": 0.99941504, "index": 6, "word": "年", "start": 5, "end": 6}, {"entity": "COMMACATEGORY", "score": 0.9985468, "index": 7, "word": "，", "start": 6, "end": 7}, {"entity": "DF", "score": 0.96427774, "index": 8, "word": "益", "start": 7, "end": 8}, {"entity": "VH1", "score": 0.9870613, "index": 9, "word": "驕", "start": 8, "end": 9}, {"entity": "PERIODCATEGORY", "score": 0.9976053, "index": 10, "word": "。", "start": 9, "end": 10}, {"entity": "NB1", "score": 0.9978642, "index": 11, "word": "趙", "start": 10, "end": 11}, {"entity": "NB1", "score": 0.998

In [12]:
classifier = pipeline("token-classification", model="ckiplab/bert-base-han-chinese-ws")
classifier("太史公曰：五帝、三代之記，尚矣。")

Downloading:   0%|          | 0.00/997 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/403M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'entity': 'B',
  'score': 0.999992,
  'index': 1,
  'word': '太',
  'start': 0,
  'end': 1},
 {'entity': 'I',
  'score': 0.99998415,
  'index': 2,
  'word': '史',
  'start': 1,
  'end': 2},
 {'entity': 'I',
  'score': 0.999977,
  'index': 3,
  'word': '公',
  'start': 2,
  'end': 3},
 {'entity': 'B',
  'score': 0.999992,
  'index': 4,
  'word': '曰',
  'start': 3,
  'end': 4},
 {'entity': 'B',
  'score': 0.99999225,
  'index': 5,
  'word': '：',
  'start': 4,
  'end': 5},
 {'entity': 'B',
  'score': 0.9999914,
  'index': 6,
  'word': '五',
  'start': 5,
  'end': 6},
 {'entity': 'I',
  'score': 0.9970849,
  'index': 7,
  'word': '帝',
  'start': 6,
  'end': 7},
 {'entity': 'B',
  'score': 0.99999213,
  'index': 8,
  'word': '、',
  'start': 7,
  'end': 8},
 {'entity': 'B',
  'score': 0.9999918,
  'index': 9,
  'word': '三',
  'start': 8,
  'end': 9},
 {'entity': 'I',
  'score': 0.9974807,
  'index': 10,
  'word': '代',
  'start': 9,
  'end': 10},
 {'entity': 'B',
  'score': 0.9999275,
  'index'

Pipeline for mask prediction

In [5]:
from transformers import pipeline

# Initialize 
unmasker = pipeline('fill-mask', model='ckiplab/bert-base-han-chinese')

# Input text with [MASK]
unmasker("黎[MASK]於變時雍。")

[{'score': 0.1488576978445053,
  'token': 3696,
  'token_str': '民',
  'sequence': '黎 民 於 變 時 雍 。'},
 {'score': 0.08596460521221161,
  'token': 2433,
  'token_str': '庶',
  'sequence': '黎 庶 於 變 時 雍 。'},
 {'score': 0.027848150581121445,
  'token': 3694,
  'token_str': '氏',
  'sequence': '黎 氏 於 變 時 雍 。'},
 {'score': 0.023678062483668327,
  'token': 782,
  'token_str': '人',
  'sequence': '黎 人 於 變 時 雍 。'},
 {'score': 0.018718378618359566,
  'token': 4495,
  'token_str': '生',
  'sequence': '黎 生 於 變 時 雍 。'}]

### [CKIP Transformers](https://github.com/ckiplab/ckip-transformers)

- POS tagging, NER 都有

##### set environment & model

In [ ]:
!pip install -U ckip-transformers

In [ ]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

# Initialize drivers
ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")

##### sample

In [ ]:
# Pack word segmentation and part-of-speech results
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
   assert len(sentence_ws) == len(sentence_pos)
   res = []
   for word_ws, word_pos in zip(sentence_ws, sentence_pos):
      res.append(f"{word_ws}({word_pos})")
   return "\u3000".join(res)

# Input text
text = [
   "太史公曰：五帝、三代之記，尚矣。",
   "自殷以前諸侯不可得而譜，周以來乃頗可著。",
   "孔子因史文次春秋，紀元年，正時日月，蓋其詳哉。",
   "至於序尚書則略，無年月；或頗有，然多闕，不可錄。"
]

# Run pipeline
ws  = ws_driver(text)
pos = pos_driver(ws)
ner = ner_driver(text)

# Show results
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
   print(sentence)
   print(pack_ws_pos_sentece(sentence_ws, sentence_pos))
   for entity in sentence_ner:
      print(entity)
   print()

##### main

In [ ]:
# Pack ner and pos results (xlamr format)
def pack_ws_pos_4xlamr(sentence_ws, sentence_pos) :
    assert len(sentence_ws) == len(sentence_pos)
    tokens = []
    pos = []
    for word_ws, word_pos in zip(sentence_ws, sentence_pos):
        tokens.append(word_ws)
        pos.append(word_pos)

    return tokens, pos

def ner_token_select_by_id(tokens, sentence_ner) :
    # print()
    # print("TOKENS>>>", tokens)
    # print("SENT_NER>>>", sentence_ner)
    ner = ["O" for i in range(len(tokens))]

    for entity in sentence_ner:
        counting_token_length = 0
        for t, token in enumerate(tokens) :
            tmp = (counting_token_length, counting_token_length + len(token))
            counting_token_length = counting_token_length + len(token)
            # print("\tTMP>>>", tmp)
            if tmp[0] >= entity.idx[0] :
                ner[t] = entity.ner
            if tmp[1] >= entity.idx[1] :
                break
    # print("NER>>>", ner)

    new_ner = [ne for ne in ner]
    pre_ne = ""
    next_ne = ""
    for n, ne in enumerate(ner) :
        # print("NE: ", ne)
        if n < len(ner)-1 :
            next_ne = ner[n+1]
        else :
            next_ne = ""
        if n > 0 :
            pre_ne = ner[n-1]
        
        # print("\tNEXT_NE: ", next_ne)
        # print("\tPRE_NE: ", pre_ne)

        if ne != "O" :
            if pre_ne != ne and next_ne != ne:
                new_ner[n] = "S-" + ne
            elif pre_ne != ne and next_ne == ne :
                new_ner[n] = "B-" + ne
            elif pre_ne == ne and next_ne != ne :
                new_ner[n] = "E-" + ne
            elif pre_ne == ne and next_ne == ne :
                new_ner[n] = "I-" + ne
            else :
                AssertionError("NE>>", ne)
        else :
            new_ner[n] = ne
    print("new_ner>>>", new_ner)
    return new_ner
    

input_file = open("input.txt", "r", encoding="utf-8").read()
input_sents = input_file.split("\n")
sent_dummy = ""

# Run pipeline
ws  = ws_driver(input_sents)
pos = pos_driver(ws)
ner = ner_driver(input_sents)

i = 0
# Show results
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(input_sents, ws, pos, ner):
    i = i + 1
    sent_id = str(i)
    output_dict = {"tokens":[], "lemmas":[], "pos_tags":[], "ner_tags":[]}
    sent_dummy = sent_dummy + "# ::id sents_id."+ sent_id + "\n# ::snt " + sentence + "\n"
    
    tokens, pos = pack_ws_pos_4xlamr(sentence_ws, sentence_pos)
    output_dict["tokens"] = tokens
    output_dict["lemmas"] = tokens
    print("TOKENS>>", tokens)
    print("POS>>", pos)
    output_dict["pos_tags"] = pos
    ner = ner_token_select_by_id(tokens, sentence_ner)
    output_dict["ner_tags"] = ner
 
    for feature in output_dict :
        write_txt = "# ::" + feature + " " + str(output_dict[feature])
        sent_dummy = sent_dummy + write_txt + "\n"

    sent_dummy = sent_dummy + "(d / dummy)\n\n"
    while "'" in sent_dummy :
        sent_dummy = sent_dummy.replace("'", '"')
    while '""""' in sent_dummy :
        sent_dummy = sent_dummy.replace('""""', '""')

In [ ]:
fo = open("ckip_output.txt", "w", encoding="utf-8")
fo.write(sent_dummy)
fo.close()

print("output_dict>>>\n", output_dict, "\n")
print(sent_dummy)

output_dict>>>
 {'tokens': ['臣', '竊', '不勝', '犬馬', '心', '，', '昧', '死', '願', '陛下', '詔', '有司', '，', '因', '盛夏', '吉時', '定', '皇子', '位', '。'], 'lemmas': ['臣', '竊', '不勝', '犬馬', '心', '，', '昧', '死', '願', '陛下', '詔', '有司', '，', '因', '盛夏', '吉時', '定', '皇子', '位', '。'], 'pos_tags': ['Na', 'VC', 'D', 'Na', 'Na', 'COMMACATEGORY', 'VH', 'VH', 'VK', 'Nh', 'Na', 'Na', 'COMMACATEGORY', 'Cbb', 'Nd', 'Nd', 'VC', 'Na', 'Na', 'PERIODCATEGORY'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']} 

# ::id sents_id.1
# ::snt 太史公曰：五帝、三代之記，尚矣。
# ::tokens ["太史公", "曰", "：", "五帝", "、", "三代", "之", "記", "，", "尚", "矣", "。"]
# ::lemmas ["太史公", "曰", "：", "五帝", "、", "三代", "之", "記", "，", "尚", "矣", "。"]
# ::pos_tags ["Nb", "VE", "COLONCATEGORY", "Na", "PAUSECATEGORY", "Nd", "DE", "Na", "COMMACATEGORY", "D", "T", "PERIODCATEGORY"]
# ::ner_tags ["O", "O", "O", "S-CARDINAL", "O", "S-CARDINAL", "O", "O", "O", "O", "O", "O"]
(d / dummy)

# ::id sents_id.2
# ::snt 自殷以前諸侯

------------

### [綜合工具--**UD-Kanbun**](https://github.com/KoichiYasuoka/UD-Kanbun)

- Tokenizer
- POS-Tagger
- Dependency-Parser
> ---
- UD-Kanbun 是日本教授 安岡孝一 為 漢語文言文 (Classical Chinese Texts) 設計的 NLP toolkits
- 格式參考 [Universal Dependencie (UD)](https://universaldependencies.org/#language-) 規範：
- [CoNLL-U Format](https://universaldependencies.org/format.html)

In [1]:
%%shell

pip install udkanbun
pip install spacy==2.1.0
pip install MeCab
pip install supar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.6 MB/s 
     |████████████████████████████████| 304 kB 57.6 MB/s 
     |████████████████████████████████| 574 kB 48.9 MB/s 
  Created wheel for udkanbun: filename=udkanbun-3.3.7-py3-none-any.whl size=19766860 sha256=f66273bcfa3e4efbe3dadf5917e997cff11ba9cd2b24c57266928426ec300e28
  Stored in directory: /root/.cache/pip/wheels/41/cc/87/6cd75cac454f2c860812247b3ce5abb3c184b31ce710a52815
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp37-cp37m-linux_x86_64.whl size=5626771 sha256=fa3116df5087c79acd141c688609a47860d1955c9cc855fc3f3f99d49ed04edd
  Stored in directory: /root/.cache/pip/wheels/b8/b5/8e/3da091629a21ce2d10bf90759d0cb034ba10a5cf7a01e83d64
Successfully built udkanbun ufal.udpipe
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27

In [2]:
import udkanbun
lzh=udkanbun.load()
s=lzh("不入虎穴不得虎子")
print(s,s.kaeriten(),s.to_tree(),sep="\n")

# text = 不入虎穴不得虎子
1	不	不	ADV	v,副詞,否定,無界	Polarity=Neg	2	advmod	_	Gloss=not|SpaceAfter=No
2	入	入	VERB	v,動詞,行為,移動	_	0	root	_	Gloss=enter|SpaceAfter=No
3	虎	虎	NOUN	n,名詞,主体,動物	_	4	nmod	_	Gloss=tiger|SpaceAfter=No
4	穴	穴	NOUN	n,名詞,固定物,地形	Case=Loc	2	obj	_	Gloss=cave|SpaceAfter=No
5	不	不	ADV	v,副詞,否定,無界	Polarity=Neg	6	advmod	_	Gloss=not|SpaceAfter=No
6	得	得	VERB	v,動詞,行為,得失	_	2	parataxis	_	Gloss=get|SpaceAfter=No
7	虎	虎	NOUN	n,名詞,主体,動物	_	8	nmod	_	Gloss=tiger|SpaceAfter=No
8	子	子	NOUN	n,名詞,人,関係	_	6	obj	_	Gloss=child|SpaceAfter=No


不㆑入㆓虎穴㆒不㆑得㆓虎子㆒

不 <════╗   advmod
入 ═══╗═╝═╗ root
虎 <╗ ║   ║ nmod
穴 ═╝<╝   ║ obj
不 <════╗ ║ advmod
得 ═══╗═╝<╝ parataxis
虎 <╗ ║     nmod
子 ═╝<╝     obj



In [3]:
import deplacy
deplacy.serve(s,port=None)

In [3]:
lzh2 = udkanbun.load(BERT="guwenbert-base") #guwenbert-base||guwenbert-large

 [==================================================] 100% 00:51   


Some weights of the model checkpoint at ethanyt/guwenbert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
s2 = lzh2("太史公曰：五帝、三代之記，尚矣。自殷以前諸侯不可得而譜，周以來乃頗可著。孔子因史文次春秋，紀元年，正時日月，蓋其詳哉。至於序尚書則略，無年月；或頗有，然多闕，不可錄。故疑則傳疑，蓋其慎也。")
print(s2,s2.kaeriten(),s2.to_tree(),sep="\n")

INFO:supar:Loading the data
INFO:supar:
Dataset(n_sentences=6, n_batches=5, n_buckets=5)
INFO:supar:Making predictions on the dataset


IndexError: ignored

--------------

## Core NLP 相關任務解說：



### CoreNLP


- #### Tokenization
> *Tokenization is the process of turning text into tokens. For instance, the sentence Marie was born in Paris. would be tokenized as the list "Marie", "was", "born", "in", "Paris", ".". CoreNLP splits texts into tokens with an elaborate collection of rules, designed to follow UD 2.0 specifications.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/tokenize.html)

- #### Constituency parsing
> *Provides full syntactic analysis, minimally a constituency (phrase-structure tree) parse of sentences. If a rule-based conversion from constituency parses to dependency parses is available (this is currently the case for English and Chinese, only), then a dependency representation is also generated using this conversion. The constituent-based output is saved in TreeAnnotation. We generate three dependency-based outputs, as follows: basic dependencies, saved in BasicDependenciesAnnotation; enhanced dependencies saved in EnhancedDependenciesAnnotation; and enhanced++ dependencies in EnhancedPlusPlusDependenciesAnnotation. Most users of our parser will prefer the latter representation. Constituency parsers internally generate binary parse trees, which can also be saved.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/parse.html)

- #### Dependency parsing
> *Provides a fast syntactic dependency parser. We generate three dependency-based outputs, as follows: basic, uncollapsed dependencies, saved in BasicDependenciesAnnotation; enhanced dependencies saved in EnhancedDependenciesAnnotation; and enhanced++ dependencies in EnhancedPlusPlusDependenciesAnnotation. Most users of our parser will prefer the latter representation.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/depparse.html)


- #### Part-of-Speech tagging
> *Part of speech tagging assigns part of speech labels to tokens, such as whether they are verbs or nouns. Every token in a sentence is applied a tag. For instance, in the sentence Marie was born in Paris. the word Marie is assigned the tag NNP.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/pos.html)



- #### Lemmatization
> *Lemmatization maps a word to its lemma (dictionary form). For instance, the word was is mapped to the word be.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/lemma.html)



- #### Named Entity Recognition (NER)
> *Recognizes named entities (person and company names, etc.) in text. Principally, this annotator uses one or more machine learning sequence models to label entities, but it may also call specialist rule-based components, such as for labeling and interpreting times and dates. Numerical entities that require normalization, e.g., dates, have their normalized value stored in NormalizedNamedEntityTagAnnotation. For more extensive support for rule-based NER, you may also want to look at the RegexNER annotator. The set of entities recognized is language-dependent, and the recognized set of entities is frequently more limited for other languages than what is described below for English. As the name “NERClassifierCombiner” implies, commonly this annotator will run several named entity recognizers and then combine their results but it can run just a single annotator or only rule-based quantity NER.* <br>
> *For English, by default, this annotator recognizes named (PERSON, LOCATION, ORGANIZATION, MISC), numerical (MONEY, NUMBER, ORDINAL, PERCENT), and temporal (DATE, TIME, DURATION, SET) entities (12 classes). Adding the regexner annotator and using the supplied RegexNER pattern files adds support for the fine-grained and additional entity classes EMAIL, URL, CITY, STATE_OR_PROVINCE, COUNTRY, NATIONALITY, RELIGION, (job) TITLE, IDEOLOGY, CRIMINAL_CHARGE, CAUSE_OF_DEATH, (Twitter, etc.) HANDLE (12 classes) for a total of 24 classes. Named entities are recognized using a combination of three CRF sequence taggers trained on various corpora, including CoNLL, ACE, MUC, and ERE corpora. Numerical entities are recognized using a rule-based system.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/ner.html)


- #### Coreference Resolution
> *The CorefAnnotator finds mentions of the same entity in a text, such as when “Theresa May” and “she” refer to the same person. The annotator implements both pronominal and nominal coreference resolution. The entire coreference graph (with head words of mentions as nodes) is saved as a CorefChainAnnotation.*<br>
> [--CoreNLP](https://stanfordnlp.github.io/CoreNLP/coref.html)

...

### Others
- #### Abstract Meaning Representation parsing (AMR)
> **<br>
> [--]()

- #### Semantic Role Labeling (SRL)
> **<br>
> [--]()
...

In [ ]:
%%shell
wget http://bollin.inf.ed.ac.uk/public/direct/amreager/resources_zh.tar.gz
tar -xf resources_zh.tar.gz
rm -f resources_zh.tar.gz

In [ ]:
%%shell

tar -xf CHINESE.tar.gz
rm -f CHINESE.tar.gz